<h2>개인 구글 드라이브와 colab 연동</h2>

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


<h2>"SMSSpamCollection" 데이터를 읽고 문장과 정답을 분리하여 각 리스트에 저장</h2>

<pre>
<b>1. 데이터의 형태(SMSSpamCollection)</b>
  라벨(스팸 또는 햄) \t(tab) 문장 
  
  위와 같은 형태로 저장되어 있음
  
  예시)
    ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    spam\tCustomer service annoncement. You have a New Years delivery waiting for you. Please call 07046744435 now to arrange delivery
    ...
  
  따라서 입력 데이터를 읽고 \t을 기준으로 입력 문장을 분리한 후에 문장과 라벨을 각각 x_data, y_data 리스트에 저장

<b>2. x_data, y_data 형태</b>
  x_data = [ 문장1, 문장2, 문장3, ... ]
  y_data = [ 문장1의 라벨, 문장2의 라벨, 문장3의 라벨, ... ]
</pre>

In [2]:
file_path = "/gdrive/My Drive/인공지능 실습/week5/SMSSpamCollection"

# 파일 읽기
with open(file_path,'r',encoding='utf8') as inFile:
  lines = inFile.readlines()

x_data, y_data = [], []
for line in lines:
  pieces = line.strip().split('\t')
  label, sentence = pieces[0], pieces[1]
  
  x_data.append(sentence)
  y_data.append(label)
  
print("x_data의 개수 : " + str(len(x_data)))
print("y_data의 개수 : " + str(len(y_data)))

x_data의 개수 : 1500
y_data의 개수 : 1500


<h2>Tokenizer 라이브러리를 사용하여 입력 문장을 index로 치환</h2>

<pre>
<b>1. tokenizer.fit_on_texts(data) 함수를 이용하여 각 단어를 index로 치환하기 위한 딕셔너리 생성</b>
   생성된 딕셔너리는 tokenizer 객체 안에 저장됨
  
  tokenizer.fit_on_texts(data)
  args
    data : 문자열 element를 가지고 있는 리스트
  return
    X
    
  딕셔너리 예시)
    {'to': 1, 'i': 2, 'you': 3, 'a': 4, 'the': 5, 'and': 6, 'for': 7 ... }
    
<b>2. tokenizer.texts_to_matrix(data) 함수를 이용하여 각 단어를 one-hot 벡터로 표현하며 문장은 각 단어의 one-hot 벡터의 합으로 표현</b>

  tokenizer.texts_to_sequences(data)
  args
    data : 문자열 element를 가지고 있는 리스트
  return : 
    indexing 된 리스트

  예시)
    전체 단어 사전 : {'i': 1, 'am': 2, 'happy': 3, 'sad': 4}
    i     : [1, 0, 0, 0]
    am    : [0, 1, 0, 0]
    happy : [0, 0, 1, 0]
    sad   : [0, 0, 0, 1]
    
    문장 1 : i am happy   -> [1, 1, 1, 0]
    문장 2 : i am sad     -> [1, 1, 0, 1]
    문장 3 : i am sad sad -> [1, 1, 0, 2]
    
  indexing 예시)
    x_data indexing 하기 전 : Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    x_data indexing 하기 후 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    y_data indexing 하기 전 : ham
    y_data indexing 하기 후 : 1
</pre>

In [4]:
from keras.preprocessing.text import Tokenizer


# spam, ham 라벨을 대응하는 index로 치환하기위한 딕셔너리
label2index = {'spam':0, 'ham':1}
index2label = {0:"spam", 1:"ham"}

# indexing 한 데이터를 넣을 리스트 선언
indexing_x_data, indexing_y_data = [], []

for label in y_data:
  indexing_y_data.append(label2index[label])

tokenizer = Tokenizer(num_words=300)

# x_data를 사용하여 딕셔너리 생성
tokenizer.fit_on_texts(x_data)                

# x_data에 있는 각 문장들을 one-hot 벡터의 합으로 치환하고 그 결과값을 indexing_x_data에 저장
indexing_x_data = tokenizer.texts_to_matrix(x_data, mode='count').tolist()

print("x_data indexing 전 : " + str(x_data[0]))
print("x_data indexing 후 : " + str(indexing_x_data[0]))
print("y_data indexing 전 : " + str(y_data[0]))
print("y_data indexing 후 : " + str(indexing_y_data[0]))

x_data indexing 전 : Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
x_data indexing 후 : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

<h2>SVM 모델 학습</h2>

<pre>    
<b>1. 입력 데이터를 9 대 1 비율로 나누어 학습, 평가에 사용</b>
  
<b>2. svm.fit(x, y) 함수를 사용하여 SVM 모델 학습</b>
  svm.fit(x, y)
  args
    x : indexing 된 문장들이 있는 리스트
    y : x의 각 문장에 대응하는 라벨이 있는 리스트
  return : 
    X
</pre>

In [5]:
from sklearn.svm import SVC


# 전체 데이터를 9:1의 비율로 나누어 학습 및 평가 데이터로 사용
number_of_train_data = int(len(indexing_x_data)*0.9)

train_x, train_y = indexing_x_data[:number_of_train_data], indexing_y_data[:number_of_train_data]
test_x, test_y = indexing_x_data[number_of_train_data:], indexing_y_data[number_of_train_data:]

print("train_x의 개수 : " + str(len(train_x)))
print("train_y의 개수 : " + str(len(train_y)))
print("test_x의 개수 : " + str(len(test_x)))
print("test_y의 개수 : " + str(len(test_y)))

svm = SVC(kernel='rbf')
svm.fit(train_x, train_y)

train_x의 개수 : 1350
train_y의 개수 : 1350
test_x의 개수 : 150
test_y의 개수 : 150


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

<h2>SVM 모델을 이용한 평가</h2>

<pre>
<b>1. svm.predict(data) 함수를 사용하여 SVM 모델을 이용하여 평가</b>
  
  svm.predict(data)
  args
    data : indexing 된 문장들이 있는 리스트
  return : 
    입력 문장들에 대한 모델의 출력 라벨 리스트
    
<b>2. 성능 측정</b>
  정답 라벨과 모델의 출력 라벨을 비교하여 성능 측정
  
<b>3. tokenizer.sequences_to_texts(data) 함수를 이용하여 indexing 된 데이터를 단어로 치환</b>

  tokenizer.sequences_to_texts(data)
  args
    data : indexing 된 리스트
  return : 
    단어로 치환된 리스트
    
  예시)
    [38, 93, 239, 240, 241, 242, 53, 11, 243, 72, 94, 244, 245, 126, 246, 247, 73, 74, 248, 127] -> Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    
<b>4. 입력 문장에 대한 모델의 출력과 정답 출력</b>

</pre>

In [6]:
from sklearn.metrics import accuracy_score


predict = svm.predict(test_x)

print('Accuracy: %.2f' % accuracy_score(test_y, predict))
for index in range(len(x_data[number_of_train_data:number_of_train_data+5])):
  print()
  print("문장 : ", x_data[index])
  print("정답 : ", index2label[test_y[index]])
  print("모델 출력 : ", index2label[predict[index]])

Accuracy: 0.95

문장 :  Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
정답 :  spam
모델 출력 :  spam

문장 :  Ok lar... Joking wif u oni...
정답 :  ham
모델 출력 :  ham

문장 :  Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
정답 :  ham
모델 출력 :  ham

문장 :  U dun say so early hor... U c already then say...
정답 :  ham
모델 출력 :  ham

문장 :  Nah I don't think he goes to usf, he lives around here though
정답 :  ham
모델 출력 :  ham


<h2>과제</h2>

<pre>
<b>1. 카이제곱 통계량을 이용하여 분류할 각 클래스와 관련된 단어를 추출</b>
  
  카이제곱 통계량 수식(w, c) = ( (A + B + C + D) x ((AD-BC) x (AD-BC)) ) / ( (A+B) x (A+C) x (B+D) x (C+D) )

  이 때, 
    A = 클래스 c에 대응하는 문서 중에서 단어 w를 포함하고 있는 문서의 수
    B = 클래스 c 이외의 문서 중에서 단어 w를 포함하고 있는 문서의 수
    C = 클래스 c에 속한 문서 중에서 단어 w를 포함하고 있지 않은 문서의 수
    D = 클래스 c 이외의 문서 중에서 단어 w를 포함하고 있지 않은 문서의 수

  이를 사용하여 계산한 카이제곱 점수를 내림차순으로 정렬하여 상위 n개의 단어 추출하여 클래스별 자질 사전 생성
 
</pre>